In [10]:
%matplotlib inline
import numpy 
from matplotlib import pyplot as plt
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

In [11]:
#Parameters

nx = 81
dx = 0.25
dt = 0.0002
gamma = 1.4 
nt = int(0.01/dx)

In [12]:
#Define rho with the initial conditions

def rho_initial():
    
    rho = numpy.zeros(nx)
    rho[0:(nx-1)/2] = 1 #kg/m^3
    rho[(nx-1)/2:nx] = 0.125 #kg/m^3
    
    return rho

#rho = rho_initial()
#print(rho)

In [13]:
#Define u with the initial conditions

def u_initial():
    
    u = numpy.zeros(nx)
    u[0:(nx-1)/2] = 0 #m/s
    u[(nx-1)/2:nx] = 0 #m/s
    
    return u   

#u = u_initial()
#print(u)

In [14]:
#Define p with the initial conditions

def p_initial():
    
    p = numpy.zeros(nx)
    p[0:(nx-1)/2] = 100 #kN/m^2
    p[(nx-1)/2:nx] = 10 #kN/m^2
    
    return p

#p = p_initial()
#print(p)

In [16]:
e = p/((gamma-1)*rho)
e_t = e + (1/2*u**2)

NameError: name 'p' is not defined

In [17]:
def richtmyer (u, nt, dt, dx):
    un = numpy.zeros((nt, len(rho)))
    un[:] = u.copy()
    ustar = u.copy()
    
  
    
    u1 = rho
    u2 = rho*u
    u3 = rho*e_t

    for t in range(1, nt):
             
        #predictor
        f = computeF(u)
        
        ustar[:,:-1] = 0.5*(u[1:,1:] + u[:,:]) - dt/(2*dx)(f[2:, 2:] - f[:-2, :-2])
        
        #corrector
        fstar = computeF(ustar)
        
        un[t,:,:-1] = u[:-1, :] - dt/dx(fstar[1:,1:] - fstar[:-1, :-1])
        
        u = un[t,:,:].copy()
        
    return un

In [18]:
def computeF(ustar):
    
    f1 = u2
    f2 = u2**2/u1 + (gamma-1)*(u3 - 0.5*u2**2/u1)
    f3 = (u3 + (gamma-1)*(u3 - 0.5*u2**2/u1))*(u2/u1)
    
    return numpy.array([f1, f2, f3])

In [19]:
a = richtmyer(u, nt, dt, dx)

NameError: name 'u' is not defined

In [9]:
print(rho[50])
print(u[20])
print(p[20])

NameError: name 'rho' is not defined